In [42]:
from pydub import AudioSegment
torch.cuda.is_available()
mp3_audio = AudioSegment.from_file(r"15min16hz.wav", format="wav")
new_mp3_audio= mp3_audio + 10 
print(len(mp3_audio)/(1000*60))

# 12 Minutes audio breaks into 3 minutes 4 audio files (slicingis done by milliseconds)

counter_audio = 180
split_audio = [mp3_audio[:180*1000]]
for i in range(4):
    split_audio.append(mp3_audio[counter_audio*1000:(counter_audio+180)*1000])
    counter_audio += 180

count = 0
# # lets save it!
for count, audio_object in enumerate(split_audio):
    count += 1
    with open(f"{count}_15min16hz.wav", 'wb') as out_f:
        audio_object.export(out_f, format='wav')

15.775283333333334


In [43]:
import librosa
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

In [44]:
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
collection_of_text = []
for i in range(4):

    speech, rate = librosa.load(f"{i+1}_15min16hz.wav", sr=16000)

    input_values = tokenizer(speech, return_tensors='pt').input_values
    # Store logits (non-normalized predictions)
    with torch.no_grad():
        logits = model(input_values).logits

    # Store predicted id's
    predicted_ids = torch.argmax(logits, dim=-1)
    # decode the audio to generate text
    # Passing the prediction to the tokenzer decode to get the transcription
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    # transcriptions = tokenizer.decode(predicted_ids[0])
    print(transcription)
    collection_of_text.append(transcription)

print(collection_of_text)
final_complete_speech = ""

# convert batch of text into one complete sentence
for i in collection_of_text:
    final_complete_speech += i

print(final_complete_speech)

with open('recognized.txt',mode ='w') as file: 
   file.write("Recognized Speech:") 
   file.write("\n") 
   file.write(final_complete_speech) 


THIS IS THE CAL ANY H AMERICAN ENGLISH TIALECT RECORDING SCOLLECTION PRODUCED WITH FUNDING FROM THE NATIONAL NDOWMET FOR THE HUMANITIES AND THE CENTER FOR APPLIED MENGUISTICS TAKE NOBER ONE SIDE A WE MUST TALK NOW ABOUT WHAT WE ARE GOING TO DO AND PLAN ACCORDINGLY EN I WANT TO EXPRESS MY THANKS THE JODG SAMU SEE BY TE WHO IS GIVEN SO MUCH TO THIS CITY WHO IS MADE CLEAN GOVERNMEN POSSIBLE IN THIS SAITY IWE STARTED BHE FOUR THIRTY FOUR REMEMBERMY CAMPAIN OF NINETEEN TWENTY NINE WHEN I TOOD THE CITY F FO ROCKEN CONDITIONS THAT EXIFTED HERE REMEMBER WHEN I WAS NOMINATED BY THE REPUBLICAN POET IN TWENTY NINE AND DROP THE NEXT DA  I WAS O MY DAR TAN REMEMBER THAT BEREN I WAS DO TAN TING BUT IT WAS USEFUL WASN'T IT THE CHARGE OF THABOMAD WAS SO STARTLING THAT SAID  HARDLYTY SO AND JUDGE SEE VERY PROVED BEYOND ANY REASONABLE DOUBT THAT ALL THE CHARGEES ABOUT MADE WER TRUE I WONT THANK JUDG I ALSO WONE OF THANK THE GROUP OF FRIEND WO GOT THE GATHER IMPITATIONS ES IT WERE FOR MY NOMINATION I KNO

In [46]:
print(mp3_audio.frame_rate) 

16000
